In [196]:
import numpy as np
import pandas as pd
from pprint import pprint

df = pd.read_csv("Assignement1-2b.csv")
data = df.values
column_names=df.columns
training_data=df.values
df.head()

,Height,Weight,Age,Gender
0,1.798398,72.545354,29,W
1,1.791747,78.826072,30,W
2,1.768286,69.288869,28,W
3,1.794980,88.006699,22,M
4,1.730018,71.812933,29,W


In [197]:
def purity(dataset):
  classes=set(dataset[:,-1])
  classes=np.array(list(classes))
  # print(classes)
  if len(classes)==1:
    return True
  return False

In [198]:
#checking purity of class
# purity(df[df['Gender']=='W'].values)
# # df[df['Gender']=='W'].values
def classify_treedata(dataset):
  classes=dataset[:,-1]
  unique_classes,count=np.unique(classes,return_counts=True)
  # print(np.unique(classes,return_counts=True))
  classification=unique_classes[np.argmax(count)]
  # print(classification)
  return classification
# classify_treedata(df[(df.Age>30) & (df.Weight>90)].values)

In [199]:
#get the splpit data left and right to decission node
def get_splits(data):
  splits={}
  _,columns=data.shape
  for column in range(columns-1): 
    splits[column]=[]
    values=data[:,column]
    unique_values=np.sort(np.unique(values))
    # print(len(unique_values))
    for index in range(len(unique_values)):
      if(index!=0):
        splits[column].append((unique_values[index-1]+unique_values[index])/2)
  # print(len(splits[2]))
  return splits

In [200]:
def get_splitdata(dataset, column, value):
  split_data=dataset[:,column]
  data_left=dataset[split_data<=value]
  data_right=dataset[split_data>value]
  return data_left,data_right

In [201]:
# column=0
# value=1.697077955
# get_splitdata(training_data,0,1.697077955)

In [202]:

def entropy_cal(dataset):
  classes=dataset[:,-1]
  _,count=np.unique(classes,return_counts=True)
  prob=count/sum(count)
  entropy=sum(prob * - np.log2(prob))
  return entropy

In [203]:
# data_left,data_right=get_splitdata(training_data,0,1.697077955)
# entropy_cal(data_right) 

In [204]:
def total_entropy(data_left, data_right):
  prob_left=len(data_left)/(len(data_left)+len(data_right))
  prob_right=len(data_right)/(len(data_left)+len(data_right))
  total_entropy= (prob_left *entropy_cal(data_left))+ (prob_right *entropy_cal(data_right))
  return total_entropy 

In [205]:
def information_gain(dataset,data_left,data_right):
  return entropy_cal(dataset)-(total_entropy(data_left,data_right))

In [206]:

def get_best_split_gain(dataset,splits):
  best_infogain=0
  for index in splits:
    for value in splits[index]:
      data_left,data_right=get_splitdata(dataset,index,value)
      # best_column=index
      # best_value=value
      info_gain=information_gain(dataset[:,(index,-1)],data_left,data_right)
      if info_gain>best_infogain:
        best_infogain=info_gain
        best_column=index
        best_value=value
  return best_column,best_value

In [207]:
splits = get_splits(training_data)
get_best_split_gain(training_data,splits)

(1, 80.03158404)

In [208]:

def DecissionTreeClassification(data,min_samples, max_depth,iteration=0):
    if (purity(data)) or (len(data) < min_samples) or (iteration == max_depth):
        classification = classify_treedata(data)
        return classification   
    else:    
        iteration += 1
        splits = get_splits(data)
        best_column, best_value = get_best_split_gain(data, splits)
        data_left, data_right = get_splitdata(data, best_column, best_value)
        # decissionrule=print(f'{column_names[best_column]} <= {best_value}')
        decissionrule=""+str(column_names[best_column])+" <= " + str(best_value) 
        sub_tree = {decissionrule: []}
        left_subtree = DecissionTreeClassification(data_left, min_samples, max_depth,iteration)
        right_subtree = DecissionTreeClassification(data_right, min_samples, max_depth,iteration)
        sub_tree[decissionrule].append(left_subtree)
        sub_tree[decissionrule].append(right_subtree)
        return sub_tree

In [209]:
min_samples=2
tree=DecissionTreeClassification(training_data, min_samples,2)
pprint(tree)

{'Weight <= 80.03158404': [{'Weight <= 70.379599155': ['W', 'W']},
                           {'Weight <= 85.44121835': ['M', 'M']}]}


In [210]:
def predict_gender(test_data,tree):
  # DecissionTreeClassification(training_data,max_depth)
  decissionrule = list(tree.keys())[0]
  #split based on spaces
  feature_name, comparison_operator, value = decissionrule.split()
  if test_data[feature_name] <= float(value):
    answer = tree[decissionrule][0]
  else:
    answer = tree[decissionrule][1]
  #if not dictionary we have reached the classification
  if not isinstance(answer, dict):
      return answer    
  else:
      sub_tree = answer
      return predict_gender(test_data, sub_tree)

In [211]:
#testing first value of training data
print(f'The predicted gender of{df.iloc[0]} is {predict_gender(df.iloc[0], tree)}')

The predicted gender ofHeight     1.798398
Weight    72.545354
Age              29
Gender            W
Name: 0, dtype: object is W


2c . Divide the data set from Question 2c) in Homework 1 (the large training data set) into a training
set comprising the first 90 data points and a test set consisting of the last 30 data elements. Use
the resulting training set to derive trees of depths 1 - 8 and evaluate the accuracy of the resulting
trees for the 90 training samples and for the test set containing the last 30 data items. Compare the
classification accuracy on the test set with the one on the training set for each tree depth. For which
depths does the result indicate overfitting ?

In [212]:
def accuracy(data,tree):
  correct=0
  for i in range(len(data)):
      if(predict_gender(data.iloc[i],tree)==data.iloc[i].Gender):
        correct+=1
  return (correct/len(data))*100

In [213]:
train_data=df[:90]
test_data=df[90:]
print(f'Total elements in train Data now : {len(train_data)}')
print(f'Total elements in test Data now : {len(test_data)}')
depth=list(range(1,13))
#print(depth)
min_samples=2
train_accuracy=[]
test_accuracy=[]
for i in depth:
  tree=DecissionTreeClassification(train_data.values, min_samples,i)
  print('------------------------------------------------------------')
  print(f'Decission Tree for Depth {i}')
  pprint(tree)
  train_accuracy.append(accuracy(train_data,tree))
  test_accuracy.append(accuracy(test_data,tree))
  print(f'Train Data Accuracy for Depth {i} is  {accuracy(train_data,tree)}')
  print(f'Test Data Accuracy for Depth {i} is {accuracy(test_data,tree)}')
  print('------------------------------------------------------------')


Total elements in train Data now : 90
Total elements in test Data now : 30
------------------------------------------------------------
Decission Tree for Depth 1
{'Weight <= 78.85303006': ['W', 'M']}
Train Data Accuracy for Depth 1 is  76.66666666666667
Test Data Accuracy for Depth 1 is 60.0
------------------------------------------------------------
------------------------------------------------------------
Decission Tree for Depth 2
{'Weight <= 78.85303006': [{'Weight <= 70.379599155': ['W', 'W']},
                           {'Weight <= 84.76644657': ['M', 'M']}]}
Train Data Accuracy for Depth 2 is  76.66666666666667
Test Data Accuracy for Depth 2 is 60.0
------------------------------------------------------------
------------------------------------------------------------
Decission Tree for Depth 3
{'Weight <= 78.85303006': [{'Weight <= 70.379599155': ['W',
                                                       {'Weight <= 70.70848896': ['M',
                                  

Overfitting of Decission Tree - Tightly fitting the training data so much for getting maximum training accuracy 

In [214]:
index = train_accuracy.index(100)
test_index=test_accuracy.index(max(test_accuracy))
print(f'Overfiiting occured at Depth {index+1}')
print(f'Maximum Test Accuacy is  at Depth {test_index+1} and Accuracy is {test_accuracy[test_index]}')

Overfiiting occured at Depth 11
Maximum Test Accuacy is  at Depth 3 and Accuracy is 76.66666666666667
